In [1]:
import pandas as pd
import glob
datapath = "./Input/"
types = ["BodyWeightSquats", "PushUps"]
extension = "*.csv"



In [14]:
for cur in types:
    current_glob = glob.glob(datapath+cur+"/"+extension)
    df = pd.DataFrame()
    li = []

    for filename in current_glob:
        df = pd.read_csv(filename, index_col=None, header=0)
        df.insert(0, 'example', filename)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    print(frame.head())


                                             example  frame    NOSE.x  \
0  ./Input/BodyWeightSquats\v_BodyWeightSquats_g0...      0  0.468669   
1  ./Input/BodyWeightSquats\v_BodyWeightSquats_g0...      1  0.468662   
2  ./Input/BodyWeightSquats\v_BodyWeightSquats_g0...      2  0.468611   
3  ./Input/BodyWeightSquats\v_BodyWeightSquats_g0...      3  0.468610   
4  ./Input/BodyWeightSquats\v_BodyWeightSquats_g0...      4  0.468484   

     NOSE.y    NOSE.z    NOSE.v  LEFT_EYE_INNER.x  LEFT_EYE_INNER.y  \
0  0.069361 -0.105805  0.999897          0.474977          0.054743   
1  0.069879 -0.103847  0.999899          0.475058          0.054792   
2  0.070022 -0.096247  0.999899          0.475086          0.054803   
3  0.069943 -0.096632  0.999902          0.475113          0.054752   
4  0.069916 -0.105230  0.999905          0.475108          0.054758   

   LEFT_EYE_INNER.z  LEFT_EYE_INNER.v  ...  RIGHT_HEEL.z  RIGHT_HEEL.v  \
0         -0.130644          0.999886  ...      0.270216    